**Рубежный контроль № 2.**

**Студент: Кулькина Дарья Александровна**

**Группа: ИУ5-62Б**

**Вариант: 12**

**Загрузка датасета**

Выбираем файл kaggle.json. Как его получить читаем здесь -> https://www.kaggle.com/general/74235

Вместо загрузки через API можно скачать датасет вручную и загрузить его в менеджер файлов. В таком случае эту и следующую ячейки кода выполнять не нужно

In [ ]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Скачиваем и распаковываем датасет

In [ ]:
!kaggle datasets download -d fivethirtyeight/fivethirtyeight-comic-characters-dataset

  0% 0.00/597k [00:00<?, ?B/s]
100% 597k/597k [00:00<00:00, 53.4MB/s]


In [ ]:
!unzip -q fivethirtyeight-comic-characters-dataset.zip
!rm fivethirtyeight-comic-characters-dataset.zip
!rm marvel-wikia-data.csv

**Импорт библиотек**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error,\
                            mean_absolute_error,\
                            precision_score,\
                            recall_score,\
                            f1_score,\
                            roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

**Чтение данных**

In [ ]:
data = pd.read_csv("dc-wikia-data.csv")

Выведем первые 5 строк выборки

In [ ]:
data.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [ ]:
data.describe()

,page_id,APPEARANCES,YEAR
count,6896.000000,6541.000000,6827.000000
mean,147441.209252,23.625134,1989.766662
std,108388.631149,87.378509,16.824194
min,1380.000000,1.000000,1935.000000
25%,44105.500000,2.000000,1983.000000
50%,141267.000000,6.000000,1992.000000
75%,213203.000000,15.000000,2003.000000
max,404010.000000,3093.000000,2013.000000


Проверим размеры выборки

In [ ]:
data.shape

(6896, 13)

Выведем типы данных колонок выборки

In [ ]:
data.dtypes

page_id               int64
name                 object
urlslug              object
ID                   object
ALIGN                object
EYE                  object
HAIR                 object
SEX                  object
GSM                  object
ALIVE                object
APPEARANCES         float64
FIRST APPEARANCE     object
YEAR                float64
dtype: object

Удалим явно бесполезные категории

In [ ]:
data = data.drop(["name", "page_id", "urlslug", "FIRST APPEARANCE"], axis = 1)

Проверим наличие null-значений

In [ ]:
data.isnull().sum()

ID             2013
ALIGN           601
EYE            3628
HAIR           2274
SEX             125
GSM            6832
ALIVE             3
APPEARANCES     355
YEAR             69
dtype: int64

Заполним пропуски в данных.

In [ ]:
# Отсутствующие значения категориальных признаков возьмём из наиболее часто встречающихся
data['ID'].fillna(data['ID'].mode()[0], inplace = True)
data['ALIGN'].fillna(data['ALIGN'].mode()[0], inplace = True)
data['EYE'].fillna(data['EYE'].mode()[0], inplace = True)
data['HAIR'].fillna(data['HAIR'].mode()[0], inplace = True)
data['SEX'].fillna(data['SEX'].mode()[0], inplace = True)
data['GSM'].fillna(data['GSM'].mode()[0], inplace = True)
data['ALIVE'].fillna(data['ALIVE'].mode()[0], inplace = True)

# Отсутствующие значения числовых признаков возьмём по среднему значению всех признаков
data['APPEARANCES'] = data['APPEARANCES'].replace(0, np.NaN)
data['APPEARANCES'].fillna(data['APPEARANCES'].mean(), inplace = True)
data['YEAR'] = data['YEAR'].replace(0, np.NaN)
data['YEAR'].fillna(data['YEAR'].mean(), inplace = True)

# Проверим датасет ещё раз
data.isnull().sum()

ID             0
ALIGN          0
EYE            0
HAIR           0
SEX            0
GSM            0
ALIVE          0
APPEARANCES    0
YEAR           0
dtype: int64

Кодируем категориальные признаки

In [ ]:
# Кодируем категориальные признаки
data.apply(LabelEncoder().fit_transform)

,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR
0,2,1,3,0,2,1,1,282,4
1,2,1,3,0,2,1,1,281,51
2,2,1,4,3,2,1,1,280,24
3,1,1,4,16,2,1,1,279,52
4,2,1,3,0,2,1,1,278,5
...,...,...,...,...,...,...,...,...,...
6891,1,1,3,0,0,1,1,23,55
6892,1,1,3,0,2,1,1,23,55
6893,1,1,3,0,2,1,1,23,55
6894,1,1,3,0,2,1,1,23,55


In [ ]:
# One Hot Encoding. Допустим, категориальный признак принимает значения {0; 1; 2}. В таком случае
# к датасету будут добавлены столбцы, представляющие значения этого признака при =0, =1 и =2. При
# равенстве признака одному из этих значений, в соответствующий столбец записывается единица, а в
# остальные - ноль
data = pd.get_dummies(data)
data.shape

(6896, 51)

Масштабируем данные

In [ ]:
# Масштабируем данные
data[["APPEARANCES", "YEAR"]] = MinMaxScaler().fit_transform(data[["APPEARANCES", "YEAR"]])

Посмотрим результат предварительной обработки

In [ ]:
data.apply(lambda x: len(x.unique()))

APPEARANCES                   283
YEAR                           80
ID_Identity Unknown             2
ID_Public Identity              2
ID_Secret Identity              2
ALIGN_Bad Characters            2
ALIGN_Good Characters           2
ALIGN_Neutral Characters        2
ALIGN_Reformed Criminals        2
EYE_Amber Eyes                  2
EYE_Auburn Hair                 2
EYE_Black Eyes                  2
EYE_Blue Eyes                   2
EYE_Brown Eyes                  2
EYE_Gold Eyes                   2
EYE_Green Eyes                  2
EYE_Grey Eyes                   2
EYE_Hazel Eyes                  2
EYE_Orange Eyes                 2
EYE_Photocellular Eyes          2
EYE_Pink Eyes                   2
EYE_Purple Eyes                 2
EYE_Red Eyes                    2
EYE_Violet Eyes                 2
EYE_White Eyes                  2
EYE_Yellow Eyes                 2
HAIR_Black Hair                 2
HAIR_Blond Hair                 2
HAIR_Blue Hair                  2
HAIR_Brown Hai

Заметим, что в датасете категориальный признак "пол" имеет 4 значения. Удалим 3 из них и приведём задачу к бинарной классификации.

In [ ]:
data = data.drop(["SEX_Genderless Characters", "SEX_Male Characters", "SEX_Transgender Characters"], axis = 1)

Извлекаем из датасета столбец с целевым признаком для классификации

In [ ]:
x = data.drop('SEX_Female Characters', axis = 1)
y = data['SEX_Female Characters']

print(x.shape)
print(y.shape)

(6896, 47)
(6896,)


Делим датасет на обучающую и тестовую выборки

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(6206, 47)
(6206,)
(690, 47)
(690,)


Обучаем модель

In [ ]:
model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('precision', precision_score(y_test, y_pred))
print('recall', recall_score(y_test, y_pred))
print('f1', f1_score(y_test, y_pred))
print('roc_auc', roc_auc_score(y_test, y_pred))

precision 0.7222222222222222
recall 0.12682926829268293
f1 0.2157676348547718
roc_auc 0.553105355795826


В качестве метрик качества для классификации будем использовать `precision`, `recall`, `f1` и `roc_auc`. Заметим, что значения метрик recall и f1 говорят о низкой точности модели, несмотря на достаточно хороший результат по метрике precision. Возможно, SVC не является оптимальной моделью для поставленной задачи.

Извлекаем из датасета столбец с целевым признаком для регрессии

In [ ]:
x = data.drop('APPEARANCES', axis = 1)
y = data['APPEARANCES']

print(x.shape)
print(y.shape)

(6896, 47)
(6896,)


Делим датасет на тестовую и обучающую выборки

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(6206, 47)
(6206,)
(690, 47)
(690,)


Обучаем модель

In [ ]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('MSE', mean_squared_error(y_test, y_pred))
print('MAE', mean_absolute_error(y_test, y_pred))

MSE 0.0007349901501398648
MAE 0.009267627592538494


Значения метрик говорят о высокой точности модели.